In [8]:
type Nm = String

data Ty = TV Nm     -- type variable
        | INT       -- integer type
        | Ty :-> Ty -- function type
      deriving (Eq,Ord,Show)

일치화 규칙

$\displaystyle
[\texttt{INT}=\texttt{INT},~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[u_2=u_2',~\cdots~],\sigma \\
[t_1\,\text{:->}\,s_1=t_2\,\text{:->}\,s_2,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[t_1=t_2,s_1=s_2,u_2=u_2',~\cdots~],\sigma \\
[x=x,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[u_2=u_2',~\cdots~],\sigma \\
[x=t,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
\{x\mapsto t\}\,[u_2=u_2',~\cdots~],\{x\mapsto t\}\uplus\sigma \qquad (x~\text{does not occur in}~ t) \\
[t=x,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[x=t,u_2=u_2',~\cdots~],\sigma
$

$\{x\mapsto t\}\,[u_1=s_1,\cdots,u_n=s_n] = \big[\{x\mapsto t\}\,u_1=\{x\mapsto t\}\,s_1,~\cdots,~\{x\mapsto t\}\,u_1=\{x\mapsto t\}\,s_1\big]$

$\{x\mapsto t\}\uplus\{x_1\mapsto t_1,~\cdots~,x_n\mapsto t_n \} = \big\{x\mapsto t, x_1\mapsto \{x\mapsto t\}t_1,~\cdots~,x_n\mapsto \{x\mapsto t\}t_n \big\}
$

In [9]:
occurs :: Nm -> Ty -> Bool
occurs x (TV x') = x==x'
occurs x INT = False
occurs x (t1 :-> t2) = occurs x t1 || occurs x t2

In [10]:
subst (x,t) ty@(TV y)
            | x == y    = t
            | otherwise = ty
subst (x,t) INT         = INT
subst (x,t) (t1 :-> t2) = subst (x,t) t1 :-> subst (x,t) t2

In [11]:
subst ("x",INT:->INT) (TV "x" :-> TV "x")

(INT :-> INT) :-> (INT :-> INT)

In [12]:
type Eqn = (Ty,Ty)     -- t1 = t2
type Subst = [(Nm,Ty)]

-- 일치화 알고리듬을 위 규칙에 따라 작성
unif :: ([Eqn],Subst) -> ([Eqn],Subst)
unif ((INT    , INT    ): es, s) = unif (es, s)
unif ((t1:->s1, t2:->s2): es, s) = unif ((t1,t2):(s1,s2):es, s)
unif ((TV x   , TV x'  ): es, s) | x==x' = unif (es, s)
unif ((TV x   , t      ): es, s) | not(x `occurs` t) = unif (es', (x,t):s')
  where es' = [(subst (x,t) t1, subst (x,t) t2) | (t1,t2)<-es]
        s' = [(x',subst (x,t) t') | (x',t')<-s]
unif ((t      , TV x   ): es, s) = unif ((TV x,t):es, s)
unif (es, s) = (es, s)

In [13]:
unif ([( (INT :-> TV "x") :-> INT , TV "y" :-> TV "x")],  [])

([],[("x",INT),("y",INT :-> INT)])

In [14]:
unif ([( (INT :-> TV "x") :-> INT , TV "x" :-> TV "y")],  [])

([(TV "x",INT :-> TV "x"),(INT,TV "y")],[])